In [100]:
#!pip install mypy
#!pip install nb_mypy

In [101]:
#%load_ext nb_mypy
%reload_ext nb_mypy
%nb_mypy On
%nb_mypy DebugOff

Version 1.0.5


In [102]:
import numpy as np
#import pandas as pd
#import math
#import re

#from typing import Union #never mind, don't need in this one

DIAG=False


https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html

In [103]:
def buildArrayFromLines(lines: list[str]) -> np.ndarray:
    
    tmp_arr : list[list[int]] = []
    for line in lines:
        tmp_line = []    
        
        for c in line:            
            if c == ".":
                tmp_line += [0]
            elif c == "#":
                tmp_line += [1]
            elif c == "\n":
                pass
            else:
                print("THIS SHOULDN'T HAVE HAPPENED: ",print(c),"-",line,"-")
        tmp_arr.append(tmp_line)
        
        if DIAG: print(">>",tmp_line)
    if DIAG: print(">>>")
    
    return np.array(tmp_arr)
    

In [104]:
def buildAllArrays(lines: list[str]) -> list[np.ndarray]:
    res = []
    
    accum_lines : list[str] = []
    for line in lines:
        line=line.strip()
        
        if len(line) == 0:
            arr_np = buildArrayFromLines(accum_lines)           
            res.append(arr_np)
            accum_lines = []
        else:
            accum_lines.append(line)
    
    if len(accum_lines) > 0:
        arr_np = buildArrayFromLines(accum_lines)           
        res.append(arr_np)
    
    return res

In [105]:
# OK
def parseFile(fileName:str)-> list[np.ndarray]:   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    map_list = buildAllArrays(lines)
        
    return map_list


In [106]:
# quick test, to make sure that parsing file and building arrays behaves as expected. 
# Set DIAG = True to see more
res = parseFile("test.txt")
assert len(res) == 2
assert (res[0].shape, res[1].shape) == ((7,9),(7,9))
assert (res[0].sum(),res[1].sum()) == (28,34)
assert (res[0][0].sum(),res[0][:,0].sum()) == (5,4) #first row, first column on first of tables
assert (res[1][0].sum(),res[1][:,0].sum()) == (4,5) #first row, first column on second of tables

In [107]:
def findReflectionLinesScore(arr_np:np.ndarray,bInCols:bool=True)->list[tuple[int,bool,int]] :
    """
    Scores all the possible reflection lines in either columns or in rows based on how imperfect refletion it represents
    
    Returns: list of tuples, with each tuple describing one _potential_ reflection line
        1. index of col/row immediately after the line
        2. bool denoting whether looking in columns or rows (True == looking in columns) 
        3. number of differences in reflection - 0 is a perfect reflection, 1 is a single cell that is different, etc        
    """
    res = []
    #out = -1
    arr_fwd = arr_np.copy()
    
    if bInCols:
        arr_fwd = np.transpose(arr_fwd)    
    # now we can always do it by row
    
    
    shape_X,_ = arr_fwd.shape

    
    
    for i in range(1,shape_X):
        # i means reflection line BEFORE i-th column
        
        my_len = min(i,shape_X-i)
        st_col = i-my_len
        if DIAG: print(shape_X,st_col,i,i+my_len)
        
        a=arr_fwd[st_col:i]
        b=arr_fwd[i:i+my_len][::-1]
                
        if DIAG: print(">..<",b)
            
        val=np.sum(abs(b-a))
        res.append((i,bInCols,val))
        
    return res

In [108]:
def scoreArray(arr_np:np.ndarray,tgt_diff:int) -> int:
    """
    Pick out (first) reflection lines in rows & cols that match the target impurity (0 for perfect reflection, 1+ for imperfect).
    Produce array score as per scoring rules.
    """
    cols_scores =  findReflectionLinesScore(arr_np,True)
    cols_scores_match_tgt = [x for (x,_,val) in cols_scores if val == tgt_diff]
    
    rows_scores =  findReflectionLinesScore(arr_np,False)
    rows_scores_match_tgt = [x for (x,_,val) in rows_scores if val == tgt_diff]
    
    if cols_scores_match_tgt:
        tgt_col = cols_scores_match_tgt[0]
    else:
        tgt_col = 0
    
    if rows_scores_match_tgt:
        tgt_row = rows_scores_match_tgt[0]
    else:
        tgt_row = 0
    
    return tgt_col + 100*tgt_row

In [109]:
def scoreAllArrays(arrays_list:list[np.ndarray],tgt_refl_score:int)->int:
    res = 0
    for my_arr in arrays_list:
        tmp_res = scoreArray(my_arr,tgt_refl_score)
        #tmp_res = changedLinesScore(my_arr)
        res += tmp_res    
    return res

In [110]:
FILE_NAME = "test.txt"
#FILE_NAME = "test2.txt"
#FILE_NAME = "test3.txt"
#FILE_NAME = "input.txt"

In [111]:
map_list = parseFile(FILE_NAME)
resA = scoreAllArrays(map_list,0)
resB = scoreAllArrays(map_list,1)

print("Result part A:",resA)
print("Result part B:",resB)

Result part A: 405
Result part B: 400


# App: Controls for future refactoring

In [112]:
def wrapperFileToBothRes(my_filename:str) -> tuple[int,int]:
    my_map_list = parseFile(my_filename)
    return scoreAllArrays(my_map_list,0),scoreAllArrays(my_map_list,1)

assert wrapperFileToBothRes("test2.txt") == (5,300)
assert wrapperFileToBothRes("test3.txt") == (400,100)
assert wrapperFileToBothRes("test.txt") == (405,400)
assert wrapperFileToBothRes("input.txt") == (31956,37617)

In [139]:
from functools import partial

def wrapperFileToBothRes2(my_filename:str,scorerA = partial(scoreAllArrays,tgt_refl_score=0),
                          scorerB = partial(scoreAllArrays,tgt_refl_score=1)) -> tuple[int,int]:
    my_map_list = parseFile(my_filename)
    return scorerA(my_map_list),scorerB(my_map_list)

assert wrapperFileToBothRes2("test2.txt") == (5,300)
assert wrapperFileToBothRes2("test3.txt") == (400,100)
assert wrapperFileToBothRes2("test.txt") == (405,400)
assert wrapperFileToBothRes2("input.txt") == (31956,37617)

In [138]:
wrapperFileToBothRes2("test.txt")

(405, 400)

In [141]:
wrapperFileToBothRes2("test.txt",scorerA=scoreAllArraysOrig,scorerB=scoreAllArraysOrig)

(405, 405)

# App: Testing things out for numpy arrays

In [113]:
x = np.zeros(10).reshape((2,5))

In [114]:
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [115]:
x[1,0]=2
x[0,1]=3

In [116]:
x[::-1]

array([[2., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0.]])

In [117]:
x[:,::-1]

array([[0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 2.]])

In [118]:
z = x.copy()
z

array([[0., 3., 0., 0., 0.],
       [2., 0., 0., 0., 0.]])

In [119]:
z[0,0]=-1

In [120]:
z

array([[-1.,  3.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.]])

In [121]:
x

array([[0., 3., 0., 0., 0.],
       [2., 0., 0., 0., 0.]])

In [122]:
(7,9)+(7,9)

(7, 9, 7, 9)

In [123]:
y="#.##..##."

In [124]:
res = []
for a in y:
    if a == ".":
        res += [0]
    else:
        res += [1]
print(res)
print(np.array([res,res]))

[1, 0, 1, 1, 0, 0, 1, 1, 0]
[[1 0 1 1 0 0 1 1 0]
 [1 0 1 1 0 0 1 1 0]]


# Recreation of original, less than perfect implementation

In [131]:
def findReflectionLinesOriginal(arr_np,bInCols=True):
    res = []
    arr_fwd = arr_np.copy()
    
    if not bInCols:
        arr_fwd = np.transpose(arr_fwd)
    
    # now we can do it by column   
    _,shape_Y = arr_fwd.shape
    
    
    for i in range(1,shape_Y):
        # i means reflection line BEFORE i-th column
        
        my_len = min(i,shape_Y-i)
        st_col = i-my_len
        if DIAG: print(shape_Y,st_col,i,i+my_len)
        
        a=arr_fwd[:,st_col:i]
        b=arr_fwd[:,i:i+my_len][:,::-1]
                
        #print(b)
        
        if np.array_equal(a,b):
            res.append(i)            

    return res
        


In [142]:
def scoreArrayOrig_partA(arr_np:np.ndarray) -> int:
    cols_reflections =  findReflectionLinesOriginal(arr_np,True)        
    rows_reflections =  findReflectionLinesOriginal(arr_np,False)    
    
    if cols_reflections:
        tgt_col = cols_reflections[0]
    else:
        tgt_col = 0
    
    if rows_reflections:
        tgt_row = rows_reflections[0]
    else:
        tgt_row = 0
    
    return tgt_col + 100*tgt_row

In [143]:
def scoreAllArraysOrig(arrays_list:list[np.ndarray],scorer=scoreArrayOrig_partA)->int:    
    res = 0
    for my_arr in arrays_list:
        tmp_res = scorer(my_arr)
        #tmp_res = changedLinesScore(my_arr)
        res += tmp_res    
    return res

In [144]:
scoreAllArraysOrig(map_list)

405

In [ ]:
def scoreArrayOrig_partB(arr_np:np.ndarray) -> int:
    orig_cols_reflections =  findReflectionLinesOriginal(arr_np,True)        
    orig_rows_reflections =  findReflectionLinesOriginal(arr_np,False)    
    
    shX,shY = arr_np.shape
    for i in range(0,shX):
        for j in range(0,shY):
            arr_np[i,j]=1-arr_np[i,j]
            
            new_cols_reflections =  findReflectionLinesOriginal(arr_np,True)   
            new_rows_reflections =  findReflectionLinesOriginal(arr_np,False)   
            
            #restore for next run
            arr_np[i,j]=1-arr_np[i,j]
    with np.nditer(map_list[1],op_flags=['readwrite']) as it:
        
    for x in it:
        print(">",x)
        x[...]=2
        print(map_list[1])
        break

In [ ]:
def scoreAllArraysOrig_partB(arrays_list:list[np.ndarray])->int:    

In [152]:
with np.nditer(map_list[1],op_flags=['readwrite']) as it:
    for x in it:
        print(">",x)
        x[...]=2
        print(map_list[1])
        break

<cell>1: error: List item 0 has incompatible type "str"; expected "Sequence[Literal['aligned', 'allocate', 'arraymask', 'copy', 'config', 'nbo', 'no_subtype', 'no_broadcast', 'overlap_assume_elementwise', 'readonly', 'readwrite', 'updateifcopy', 'virtual', 'writeonly', 'writemasked']]"  [list-item]
<cell>4: error: Unsupported target for indexed assignment ("tuple[ndarray[Any, dtype[Any]], ...]")  [index]


> 2
[[2 0 0 0 1 1 0 0 1]
 [1 0 0 0 0 1 0 0 1]
 [0 0 1 1 0 0 1 1 1]
 [1 1 1 1 1 0 1 1 0]
 [1 1 1 1 1 0 1 1 0]
 [0 0 1 1 0 0 1 1 1]
 [1 0 0 0 0 1 0 0 1]]
